# Event-based Simulation with SimPy

In [1]:
import simpy as sp
import numpy as np
from typing import List, Tuple, Dict, Union

# alternatively to using np.random's distributions,
#   you can use a distribution directly from scipy.stats:
# import scipy.stats

## SimPy Basics

SimPy has a few basic concepts to let you perform event-based simulation:
1. **Environment:** think of this as "the world" in which your simulation occurs
2. **Event:** an event that occurs in the environment -- it can be a happening, from time ticking to an action occurring
3. **Process:** described by Python generator functions -- they create events and `yield` (yeet) them out to the world
4. **Timeout:** a special time-based event, that describes how long it takes for an event to occur (assumed units)
5. **Resource:** an object representing a resource that can be acquired or released by Processes (gas station with limited fuel pumps)
6. **Container:** an object representing a resource that can be produced or consumed (like electrical power or apples)
7. **Store:** an object representing a store that can replenish or sell its items (dynamic quantities)

One thing that is key about event-based simulation is that, much like embedded systems, all processes are **interruptible** by priority, meaning that processes can be at different priorities (just like real life).

If you need to yield multiple events (actions) at a time, use the `&` operator.  If you need to yield at least one event (whichever event is closest), use the `|` operator.

In [16]:
def clock(env: sp.Environment, name: str, tick: float):
    """
    generator: some clock that ticks at "tick" seconds
    
    :param env: the universe (SimPy Environment)
    :param name: the name of the clock
    :param tick: how often this clock will speak
    """
    while True:
        print(f"clock {name}, current time: {env.now:.2f}")
        yield env.timeout(tick)

In [17]:
test_env = sp.Environment()
clock_slow = clock(test_env, "slow", 2.0)
clock_fast = clock(test_env, "fast", 0.5)
clock_ultra = clock(test_env, "ultra", 0.1)

In [18]:
clock_ultra

<generator object clock at 0x000002C3D6AC4190>

In [19]:
test_env.process(clock_ultra)
test_env.process(clock_slow)
test_env.process(clock_fast)

<Process(clock) object at 0x2c3d6998040>

In [20]:
test_env.run(until=5.)

clock ultra, current time: 0.00
clock slow, current time: 0.00
clock fast, current time: 0.00
clock ultra, current time: 0.10
clock ultra, current time: 0.20
clock ultra, current time: 0.30
clock ultra, current time: 0.40
clock fast, current time: 0.50
clock ultra, current time: 0.50
clock ultra, current time: 0.60
clock ultra, current time: 0.70
clock ultra, current time: 0.80
clock ultra, current time: 0.90
clock ultra, current time: 1.00
clock fast, current time: 1.00
clock ultra, current time: 1.10
clock ultra, current time: 1.20
clock ultra, current time: 1.30
clock ultra, current time: 1.40
clock fast, current time: 1.50
clock ultra, current time: 1.50
clock ultra, current time: 1.60
clock ultra, current time: 1.70
clock ultra, current time: 1.80
clock ultra, current time: 1.90
clock slow, current time: 2.00
clock fast, current time: 2.00
clock ultra, current time: 2.00
clock ultra, current time: 2.10
clock ultra, current time: 2.20
clock ultra, current time: 2.30
clock ultra, cu

### DMV customer example

Just to show everything in context, I'll modify the [Bank example](https://simpy.readthedocs.io/en/latest/examples/bank_renege.html) to the DMV:

In [34]:
RANDOM_SEED = 42
NEW_CUSTOMERS = 60  # Total number of customers
INTERVAL_CUSTOMERS = 1.0  # Generate new customers roughly every x minutes
MIN_PATIENCE = 10  # Min. customer patience
MAX_PATIENCE = 40  # Max. customer patience


def source(
    env: sp.Environment, 
    number: int,  # number of customers to spawn
    interval: float,  # some time interval into the exponential dist.
    counter: sp.Resource,  # the DMV counter
    the_rng: np.random.Generator
):
    """Source ("the door") generates customers randomly"""
    for i in range(number):
        customer_generator = customer(env, 'Customer %02d' % i, counter, 12.0, the_rng)
        env.process(customer_generator)
        spawn_time = the_rng.exponential(interval)
        yield env.timeout(spawn_time)


def customer(
    env: sp.Environment, 
    name: str, 
    counter: sp.Resource, 
    time_request_takes: float, 
    the_rng
):
    """Customer arrives (spawned by "the door"), waits to get served and leaves after they're served."""
    arrive = env.now
    print('at time %7.2f, %s arrived' % (arrive, name))

    with counter.request() as req:
        patience = the_rng.uniform(MIN_PATIENCE, MAX_PATIENCE)
        # Wait for the counter or abort at the end of our tether
        results = yield req | env.timeout(patience)

        wait = env.now - arrive

        if req in results:
            # We got to the counter
            print('at time %7.2f, %s waited %6.2f' % (env.now, name, wait))

            tib = the_rng.exponential(time_request_takes)
            yield env.timeout(tib)
            print('at time %7.2f, %s finished their transactions' % (env.now, name))

        else:
            # We reneged
            print('at time %7.2f, %s left the line after waiting %6.2f minutes' % (env.now, name, wait))


# Setup and start the simulation
print('SIMULATION: DMV in all of its glory (time units are minutes)')
the_rng = np.random.default_rng(RANDOM_SEED)
env = sp.Environment()

# Start processes and run
counter = sp.Resource(env, capacity=2)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter, the_rng))
env.run()

SIMULATION: DMV in all of its glory (time units are minutes)
at time    0.00, Customer 00 arrived
at time    0.00, Customer 00 waited   0.00
at time    2.40, Customer 01 arrived
at time    2.40, Customer 01 waited   0.00
at time    2.68, Customer 02 arrived
at time    4.09, Customer 03 arrived
at time    4.17, Customer 04 arrived
at time    4.24, Customer 05 arrived
at time    5.98, Customer 06 arrived
at time    7.21, Customer 07 arrived
at time    7.30, Customer 08 arrived
at time    8.20, Customer 09 arrived
at time    9.45, Customer 10 arrived
at time   11.28, Customer 11 arrived
at time   11.94, Customer 12 arrived
at time   12.39, Customer 13 arrived
at time   12.57, Customer 14 arrived
at time   12.96, Customer 15 arrived
at time   13.67, Customer 16 arrived
at time   14.13, Customer 17 arrived
at time   14.48, Customer 18 arrived
at time   15.35, Customer 19 arrived
at time   16.69, Customer 20 arrived
at time   17.77, Customer 21 arrived
at time   18.91, Customer 22 arrived
at

### Sample SimPy problem

Let's perform a simulation of four electric cars (EV, electric vehicles) trying to drive to and use a battery charging station. A car will perform three actions:
1. Driving to the EV charging station (accept the driving time as an input parameter).
2. Request/acquire a charging spot.
3. Charge the battery (accept the time it takes to charge the battery as an input parameter).

Define the charging station as only having two spots, so we have a likely chance of at least one car waiting for charging to complete.
Let's use a uniform distribution between 2 and 6 miles to model the distance each car has to drive in order to reach the EV charging station. Let's also assume that the cars can drive 35 miles per hour to reach the charging station.  Let's also use a uniform distribution to model the number of minutes that each car has to charge, from 30 minutes to 90 minutes.

LPT: Don't forget the random seed!

## Handling priorities and interrupts

### Using PriorityResource
Let's bump the EV charging simulation up now, to 10 cars, and pretend that a couple of the cars have made reservations online, so they can jump the queue ([`simpy.PriorityResource`](https://simpy.readthedocs.io/en/latest/topical_guides/resources.html#priorityresource)).

**Notice:** You can see that in the above output has cars with higher priority 0 that have "jumped the queue" over cars with lower priority 1.

### Using a time-based interrupt

Interrupts in a fair way, to indicate when a driver might have just gotten a text message and needs to leave and be somewhere instead.  We can simulate this by adding a "scheduled event" timeout in addition to the charging time timeout using the "|" (or) operator:

**Notice:** Some of the cars above waited the whole charging time, others have just left because they need to go somewhere else.

### Using PreemptiveResource 

We could even say someone has a VIP card that can kick someone off of a pump ([`simpy.PreemptiveResource`](https://simpy.readthedocs.io/en/latest/topical_guides/resources.html#preemptiveresource)), though that would probably not be a fair system at all.  However, you can imagine there are simulation cases where you would want [preemption](https://en.wikipedia.org/wiki/Preemption_(computing)), such as in an embedded system that requires fast response times, networks that require resources to be shutdown for maintenance, etc. 

To catch when an interrupt occurs, we have to use a [try-except statement](https://docs.python.org/3/tutorial/errors.html#handling-exceptions).  We also have to use a while loop to re-enter the line.  Let's see it in action: 

**Notice:** We can capture if a car gives up in line, but if we extended the leave timeout then you can also see interrupted cars wait it out.

## Monitoring your SimPy simulation

There are three ways to go about monitoring a simulation, in order from easy to difficult:
1. Implement your own monitoring harness (basically, add input/output data structures (such as lists) for monitoring into your process functions and classes)
2. Patching (creating a wrapper) for Simpy `Events` ([see this link](https://simpy.readthedocs.io/en/latest/topical_guides/monitoring.html#event-tracing))
3. Patching (creating a wrapper) for SimPy `Resources` ([see this link](https://simpy.readthedocs.io/en/latest/topical_guides/monitoring.html#resource-usage))

So, we can go ahead and use a dictionary and some lists to basically make our monitoring harness.  If we want to be fancier and make it available for analysis immediately after, we can use a pandas DataFrame after.  Either way, we must think about how we want to store our data (the columns of the table, the type of information, the granularity).  Sometimes this is called a _data schema_.

So, let's go ahead and log the following information: Event time, Car ID, Car Priority, Drive Time, Available Time Left, Charge Time Left, Event

Let's see how to setup our previous code so that we can finalize it into a `pandas.DataFrame` so we can easily get analyze it:

Now, say that we want to find out the average amount of charge time left on all cars.  One of the ways we can get this is to group by the Car ID, then get the minimum charge time of each car (or last time event of each car), and then take the mean of that:

We see that for some reason, there are certain cars that did not get any charge time at all, pretty much.  If we were trying to optimize for a fair system, then we'd want to tweak our settings (perhaps get rid of the VIP provision!) until the standard deviation is much lower.  This is the power of discrete-event simulation -- we are able to run stochastic simulations that otherwise may be difficult to compute deterministically.

# Using SimPy for real-time ECE problems

Turns out that SimPy also has a RealtimeEnvironment, which lets you synchronize events with wall-clock time.

This means that we can perform the following simulations:
1. Hardware-in-the-loop testing (VERY useful in electronics, embedded systems, medical devices, networking, etc.)
2. Testing that requires human interaction (energy, sustainability, etc.)

**Let's discuss:** what might be an event-based simulation problem in your specialization